# **Explain Your Model's Decisions With LIME**

<a class="anchor" id="0.1"></a>
# **Table of Contents**

- 1	[Introduction to LIME](#1)
- 2	[Intuition behind LIME](#2)
- 3	[Python implementation of model development](#3)
- 4	[Interpret model predictions with LIME](#4)
    - 4.1 [Import LIME package](#4.1)
    - 4.2 [Create the explainer](#4.2)
    - 4.3 [Use the explainer to explain predictions](#4.3)
- 5 [References](#5)

# **1. Introduction to LIME** <a class="anchor" id="1"></a>

[Table of Contents](#0.1)


- [LIME](https://christophm.github.io/interpretable-ml-book/lime.html) stands for **Local Interpretable Model-agnostic Explanations**. LIME focuses on training local surrogate models to explain individual predictions. Local surrogate models are interpretable models that are used to explain individual predictions of black box machine learning models. Surrogate models are trained to approximate the predictions of the underlying black box model. Instead of training a global surrogate model, LIME focuses on training local surrogate models.


- LIME is model-agnostic, meaning that it can be applied to any machine learning model. The technique attempts to understand the model by perturbing the input of data samples and understanding how the predictions change.

 ![LIME](https://miro.medium.com/max/1165/1*k-rxjnvUDTwk8Jfg6IYBkQ.png)

- Model-specific approaches aim to understand the black model machine learning model by analysing the internal components and how they interact. LIME provides local model interpretability. LIME modifies a single data sample by tweaking the feature values and observes the resulting impact on the output. The most common question is probably: why was this prediction made or which variables caused the prediction.

# **2. Intuition behind LIME** <a class="anchor" id="2"></a>

[Table of Contents](#0.1)


- The intuition behind LIME is very simple. First, forget the training data and imagine we have only the black box model where we supply the input data. The black box model generate the predictions for the model. We can enquire the box as many times as we like. Our objective is to understand why the machine learning model made a certain prediction. 

- Now, [LIME](https://christophm.github.io/interpretable-ml-book/lime.html) comes into play. LIME tests what happens to the predictions when we provide variations in the data which is being fed into the machine learning model. 

- [LIME](https://christophm.github.io/interpretable-ml-book/lime.html) generates a new dataset consisting of permuted samples and the corresponding predictions of the black box model. On this new dataset LIME then trains an [interpretable model](https://christophm.github.io/interpretable-ml-book/simple.html#simple). It is weighted by the proximity of the sampled instances to the instance of interest. The learned model should be a good approximation of the machine learning model predictions locally, but it does not have to be a good global approximation. This kind of accuracy is also called local fidelity.

- A global decision boundary is likely to be non linear but if you zoom in enough, it probably is!

- Mathematically, local surrogate models with interpretability constraint can be expressed as follows:

$$explanation(x)=arg min_g∈G L(f,g,πx)+Ω(g)$$

- The explanation model for instance x is the model g (e.g. linear regression model) that minimizes loss function L (e.g. mean squared error). It measures how close the explanation is to the prediction of the original model f (e.g. an xgboost model), while the model complexity Ω(g) is kept low (e.g. prefer fewer features). G is the family of possible explanations. 

- In practice, LIME only optimizes the loss part. The user has to determine the complexity, e.g. by selecting the maximum number of features that the linear regression model may use.

- So, the recipe for training local surrogate models is as follows:

  - 1 Select your instance of interest for which you want to have an explanation of its black box prediction.
  - 2 Perturb your dataset and get the black box predictions for these new points.
  - 3 Weight the new samples according to their proximity to the instance of interest.
  - 4 Train a weighted, interpretable model on the dataset with the variations.
  - 5 Explain the prediction by interpreting the local model.

# **3. Model development** <a class="anchor" id="3"></a>

[Table of Contents](#0.1)

## **3.1 Load Preliminaries** <a class="anchor" id="3.1"></a>

[Table of Contents](#0.1)

In [ ]:
#!pip install lime

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Ignore warnings
import warnings

warnings.filterwarnings("ignore")

## **3.2 Read Data** <a class="anchor" id="3.2"></a>

[Table of Contents](#0.1)

In this workshop we will aim to predict energy ratings of building in New York City. This dataset contains 41 features of which some are probably not that useful for predicting these energy ratings.

In [ ]:
# Read and preview data
df = pd.read_csv("data/NYC_Energy_Water_Data.csv").replace(
    {"Not Available": np.nan, "Not found": np.nan}
)
df.head()

## **3.3 View Summary of data** <a class="anchor" id="3.3"></a>

[Table of Contents](#0.1)


Uncomment the cell below to see a description of every feature.

In [ ]:
# description = pd.read_csv('data/Description.csv')
# for i in range(len(description)):
#     print(description.loc[i].values)

In [ ]:
df.info()

- Here are all the variables which we will use:

  - 1 `Largest Property Use Type - Gross Floor Area (ft²)` - Floor area, there can be multiple property types (hotel/office/...) per building, we only consider the largest one.
  - 2 `Year Built` - This is the year in which your property was constructed. If your property has undergone a complete renovation that included gutting and rebuilding the interior, then you can indicate the date of this renovation as the year built.
  - 3 `Occupancy` - The percentage of your property’s Gross Floor Area (GFA) that is occupied and operational.
  - 4 `Weather Normalized Site EUI (kBtu/ft²)` - Energy use intensity as calculated at the property site in kBtus per gross square foot (kBtu/ft2) for the reporting year, normalized for weather.
  - 5 `Weather Normalized Site Electricity Intensity (kWh/ft²)` - Weather Normalized Site Energy divided by property size or by flow through a water/wastewater treatment plant.
  - 6 `Weather Normalized Site Natural Gas Intensity (therms/ft²)` - Weather Normalized Site Energy divided by property size or by flow through a water/wastewater treatment plant.
  - 7 `Weather Normalized Site Natural Gas Use (therms)` - The energy use your property would have consumed during 30-year average weather conditions
  - 8 `Electricity Use - Grid Purchase (kBtu)` - Energy Use by Type is a summary of the annual consumption of an individual type of energy. Annual totals are available for Electricity Use - Grid Purchase.
  - 9 `Weather Normalized Site Electricity (kWh)` - The energy use your property would have consumed during 30-year average weather conditions
  - 10 `Total GHG Emissions (Metric Tons CO2e)` - The total direct and indirect greenhouse gases emitted by the property, reported in metric tons of carbon dioxide equivalent (MtCO2e) for the reporting year.
  - 11 `Direct GHG Emissions (Metric Tons CO2e)` - The total direct greenhouse gases emitted by the property, reported in metric tons of carbon dioxide equivalent (MtCO2e) for the reporting year.
  - 12 `Indirect GHG Emissions (Metric Tons CO2e)` - The total indirect greenhouse gases emitted by the property, reported in metric tons of carbon dioxide equivalent (MtCO2e) for the reporting year.
  - 13 `Water Use (All Water Sources) (kgal)` - Sum of all water meters.


- The target variable is `ENERGY STAR Score`, which is a 1-to-100 percentile ranking for specified building types. A high rating is good and low is bad.

In [ ]:
# Drop rows where we have no target variable because there is nothing to predict
df = df.dropna(subset=["ENERGY STAR Score"])

# Only select numerical features (see the text cell above for descriptions).
relevant_features = [
    "Largest Property Use Type - Gross Floor Area (ft²)",
    "Year Built",
    "Occupancy",
    "ENERGY STAR Score",
    "Weather Normalized Site EUI (kBtu/ft²)",
    "Weather Normalized Site Electricity Intensity (kWh/ft²)",
    "Weather Normalized Site Natural Gas Intensity (therms/ft²)",
    "Weather Normalized Site Natural Gas Use (therms)",
    "Electricity Use - Grid Purchase (kBtu)",
    "Weather Normalized Site Electricity (kWh)",
    "Total GHG Emissions (Metric Tons CO2e)",
    "Direct GHG Emissions (Metric Tons CO2e)",
    "Indirect GHG Emissions (Metric Tons CO2e)",
    "Water Use (All Water Sources) (kgal)",
]
df = df[relevant_features].astype(float)

## **3.4 Missing values treatment** <a class="anchor" id="3.4"></a>

[Table of Contents](#0.1)

In [ ]:
df.isnull().sum()

We can see that there are quite a lot of missing values in the dataset. For convenience, we will fill them by the mean of respective columns.

In [ ]:
df = df.fillna(df.mean())

Again check for missing values.

In [ ]:
df.isnull().sum()

Now, we can see that there are no missing values in the data.

## **3.5 Feature Vector and Target Variable** <a class="anchor" id="3.5"></a>

[Table of Contents](#0.1)

In [ ]:
# Remove the target variable from the features
relevant_features.remove("ENERGY STAR Score")

In [ ]:
len(relevant_features)

In [ ]:
# Declare feature vector and target variable
X = df[relevant_features]
y = df["ENERGY STAR Score"]

## **3.6 Train-Test Split** <a class="anchor" id="3.6"></a>

[Table of Contents](#0.1)

In [ ]:
# Split the data into train and test data:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## **3.7 Build the Random Forest model** <a class="anchor" id="3.7"></a>

[Table of Contents](#0.1)

In [ ]:
# Build the model with Random Forest Regressor :
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
model.fit(X_train, y_train)

## **3.8 Generate Predictions** <a class="anchor" id="3.8"></a>

[Table of Contents](#0.1)

In [ ]:
y_pred = model.predict(X_test)

## **3.9 Evaluate Performance** <a class="anchor" id="3.9"></a>

[Table of Contents](#0.1)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred) ** (0.5)

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

# **4. Interpret model predictions with LIME** <a class="anchor" id="4"></a>

[Table of Contents](#0.1)

## **4.1 Import LIME package** <a class="anchor" id="4.1"></a>

[Table of Contents](#0.1)

In [ ]:
import lime
import lime.lime_tabular

## **4.2 Create the Explainer** <a class="anchor" id="4.2"></a>

[Table of Contents](#0.1)

In [ ]:
# LIME has one explainer for all the models
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train.values,
    feature_names=X_train.columns.values.tolist(),
    class_names=["ENERGY STAR Score"],
    verbose=True,
    mode="regression",
)

## **4.3 Use the explainer to explain predictions** <a class="anchor" id="4.3"></a>

[Table of Contents](#0.1)

- Here, we will choose 5 instances and use them to explain the predictions.

## **Select 3rd instance**

In [ ]:
# Choose the 3rd instance and use it to predict the results
j = 3
# Use just the first 5 features for this example for illustrative purposes
exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=5)

In [ ]:
# The true value of this instance
y_test.iloc[j]

In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True)

In [ ]:
# This list shows the decision rules and the impact it has on the outcome, see the values in the table on the right above here.
exp.as_list()

### **Interpretation**

- The predicted value of the energy star score is 17.90 and the true score is 21.
- The prediction is mostly driven by the `Weather Normalized Site Electricity Intensity`, and in this case negatively.
- `Weather Normalized Site EUI` and `Gross Floor Area` also negatively impact the model's decision.
- `Total GHG Emissions` and `Weather Normalized Site Natural Gas Intensity` impact the prediction positively.

## **Select 11th instance**

In [ ]:
# Choose the 10th instance and use it to predict the results
j = 11
# This time we use 10 features
exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=10)

In [ ]:
# The true value of this instance
y_test.iloc[j]

In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

### **Interpretation**

- The predicted value of the energy star score is 98.60 and the true score is 100.
- The prediction is again most strongly driven by the `Weather Normalized Site EUI`, but positively in this case.
- For example, the second most important feature: `Total GHG Emissions` is below the threshold of 261.5 (see the list) and therefore is an indicator of a well performing building in terms of energy usage. In the table (right top) the corresponding value of 42.90 is in fact lower than that threshold in the list and so we can see the reasons that our model bases its very high predicted `ENERGY STAR Score` on.
- (Remember that LIME uses local thresholds so the decision rules do not apply for every instance! Intuitively, a certain amount of gas use is not necessarily good or bad, it depends on how big the building is.)

## **Select 16th instance**

In [ ]:
# Choose the 16th instance and use it to predict the results
j = 16
# Use all features.
exp = explainer.explain_instance(
    X_test.values[j], model.predict, num_features=len(relevant_features)
)

In [ ]:
y_test.iloc[16]

In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

### **Interpretation**

- The predicted value of the energy star score is 34.4 and the true score is 50.
- In this case we use all 13 features but we clearly see that most at the bottom have little impact on the prediction. In fact, `Occupancy` has no impact at all.

## **Interpret an instance DIY**

In [ ]:
# Select a random instance and use it to predict the results
j = np.random.randint(len(X_test), size=1).item()
print(j)
# Pick an amount of features, you can always change it an run it again
n1 = 
exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=n1)

In [ ]:
# True Energy rating
...

In [ ]:
# Show the predictions
...

In [ ]:
# Show the feature threhold and rules
...

### **Interpretation**

- [Fill this in]

## **Interpret another instance DIY**

In [ ]:
# Select a random instance and use it to predict the results
j = np.random.randint(len(X_test), size=1).item()
print(j)
# Pick an amount of features, you can always change it an run it again
n2 = 
exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=n2)

In [ ]:
# True Energy rating
...

In [ ]:
# Show the predictions
...

In [ ]:
# Show the feature threhold and rules
...

### **Interpretation**

- [Fill this in]

## Some question to think about:
- 1. What is the most important feature in most (if not all) cases and why? (hint: EUI stands for Energy use intensity)
- 2. What is the impact of the amount of features used in LIME?
- 3. Can you generalize about your model's decision with LIME? Why or why not?
- 4. Does LIME only inform you about your model or might you learn something about the data and the underlying problem as well?

Write your answers down or think about them!

# **5. References** <a class="anchor" id="5"></a>

[Table of Contents](#0.1)


The work done in this kernel is based on the following resources:

- 1 https://www.kaggle.com/code/prashant111/explain-your-model-predictions-with-lime
- 2 https://www.kaggle.com/datasets/mikhailma/energy-efficiency-of-buildings-in-new-york
- 3 https://christophm.github.io/interpretable-ml-book/
- 4 https://christophm.github.io/interpretable-ml-book/lime.html
- 5 https://blog.dominodatalab.com/shap-lime-python-libraries-part-2-using-shap-lime/
- 6 https://www.analyticsvidhya.com/blog/2017/06/building-trust-in-machine-learning-models/
- 7 https://towardsdatascience.com/understanding-model-predictions-with-lime-a582fdff3a3b
- 8 https://marcotcr.github.io/lime/tutorials/Using%2Blime%2Bfor%2Bregression.html

[Go to Top](#0)